In [2]:
from app.core.model_builder import ModelBuilder
from sixgill.pipesim import Model
from sixgill.definitions import ModelComponents, Parameters
from app.project import get_string_values_from_class
from sixgill.core.model_context import ContextError
from sixgill.core.mapping import ParameterError
import pandas as pd
import logging

logger = logging.getLogger(__name__)

In [3]:
model = Model.open("trail.pips")

INFO:manta.server.manager:Starting PIPESIM server on thread ID: 19700
INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:manta.server.manager:Waiting for PIPESIM server to start.


In [4]:
df = pd.read_excel("trail_copy.xlsx", sheet_name=None)

In [5]:
flowline = df["Flowline Geometry"]

In [6]:
model.set_value(
    context="6-BFW-2", parameter=Parameters.Flowline.DETAILEDMODEL, value=True
)

In [14]:
def convert_to_detailed_flowline(flowlines: list):
    dict = {
        flowline: {Parameters.Flowline.DETAILEDMODEL: True} for flowline in flowlines
    }
    return dict

In [17]:
convert_to_detailed_flowline(flowline["Name"].unique().tolist())

{'6-BFW-2': {'DetailedModel': True}, '6-BFW-4': {'DetailedModel': True}}

In [7]:
flowline["Name"] = flowline["Name"].fillna(method="ffill")
flowline = flowline.dropna(subset=["Name"])
flowline

,Name,HorizontalDistance,MeasuredDistance,Elevation
1,6-BFW-2,0,NaN,100
2,6-BFW-2,100,NaN,120
3,6-BFW-2,200,NaN,150
4,6-BFW-2,300,NaN,200
5,6-BFW-4,1,NaN,101
6,6-BFW-4,101,NaN,121
7,6-BFW-4,201,NaN,151
8,6-BFW-4,301,NaN,201


In [8]:
def set_flowline_geometry(context, data):
    try:
        model.set_geometry(context, data)
    except ContextError as e:
        logger.error(f"Error setting flowline geometry: {e}")
    except ParameterError as e:
        logger.error(f"Error setting flowline geometry: {e}")


def import_flowline_geometry(flowline_data):
    for name, data in flowline_data.items():
        try:
            set_flowline_geometry(name, data)
        except ValueError as e:
            logger.error(f"Error importing flowline geometry: {e}")


def create_flowline_geometry_data(flowline) -> dict:
    if "Name" not in flowline.columns:
        raise ValueError("Name column is required in flowline geometry data")
    flowline["Name"] = flowline["Name"].fillna(method="ffill")
    flowline = flowline.dropna(subset=["Name"])
    flowline_data = {}
    for name, group in flowline.groupby("Name"):
        cols = set(
            get_string_values_from_class(Parameters.FlowlineGeometry)
        ).intersection(set(group.columns))
        flowline_data[name] = group[list(cols)]
    return flowline_data

In [10]:
data = create_flowline_geometry_data(flowline)

C:\Users\IDM252577\AppData\Local\Temp\ipykernel_14924\1281871392.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flowline["Name"] = flowline["Name"].fillna(method="ffill")


In [11]:
import_flowline_geometry(data)

ValueError: Flowline 6-BFW-4 is a simple flowline.